In [1]:
import json
import os
import sys
import boto3
from utils import bedrock, print_ww
from utils.utils import extract_model_info

## Setting up Bedrock API.

In [2]:
boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
    runtime=False
)

Create new client
  Using region: None
boto3 Bedrock client successfully created!
bedrock(https://bedrock.us-east-1.amazonaws.com)


### Let's take a look at which models we have access to.

In [3]:
all_models = boto3_bedrock.list_foundation_models()
print(f"extract_model_info(all_models): {extract_model_info(all_models)}")

extract_model_info(all_models): [('Titan Text Large', 'amazon.titan-tg1-large'), ('Titan Text Embeddings', 'amazon.titan-e1t-medium'), ('Titan Text Embeddings v2', 'amazon.titan-embed-g1-text-02'), ('Titan Text G1 - Express', 'amazon.titan-text-express-v1'), ('Titan Embeddings G1 - Text', 'amazon.titan-embed-text-v1'), ('Stable Diffusion XL', 'stability.stable-diffusion-xl'), ('Stable Diffusion XL', 'stability.stable-diffusion-xl-v0'), ('J2 Grande Instruct', 'ai21.j2-grande-instruct'), ('J2 Jumbo Instruct', 'ai21.j2-jumbo-instruct'), ('Jurassic-2 Mid', 'ai21.j2-mid'), ('Jurassic-2 Mid', 'ai21.j2-mid-v1'), ('Jurassic-2 Ultra', 'ai21.j2-ultra'), ('Jurassic-2 Ultra', 'ai21.j2-ultra-v1'), ('Claude Instant', 'anthropic.claude-instant-v1'), ('Claude', 'anthropic.claude-v1'), ('Claude V1 100k', 'anthropic.claude-v1-100k'), ('Claude', 'anthropic.claude-v2'), ('Claude V2 100k', 'anthropic.claude-v2-100k'), ('Command', 'cohere.command-text-v14')]


In [4]:
bedrock_runtime = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

Create new client
  Using region: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)


In [5]:
prompt_data = """
Human: Tell me everything you know about JSON schema.

Assistant:"""

In [6]:
body = json.dumps({"prompt": prompt_data, "max_tokens_to_sample": 500})
modelId = "anthropic.claude-v2" 
accept = "application/json"
contentType = "application/json"

In [7]:
response = bedrock_runtime.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())

print(response_body.get("completion"))

 Here is a summary of what I know about JSON Schema:

- JSON Schema is a specification for defining the structure and validation rules of JSON data. It provides a declarative syntax for describing JSON data formats.

- The main purposes of JSON Schema are to validate JSON data and provide clear human- and machine-readable documentation.

- JSON Schema files are themselves written in JSON format and usually have the file extension .schema.json. 

- The basic building blocks of JSON Schema are:

    - Types - Specifies the allowed data types of values like string, number, object, array, etc.

    - Properties - Describes the names and allowed values of object properties.

    - Required - Lists which object properties must be present.

    - Format - Provides validation formats for strings like date, email, etc.

    - Enum - Lists acceptable value options for a property.

- JSON Schema validation works by comparing a JSON document against the schema and checking if it adheres to the str

In [8]:
prompt_data = """
Human: Your role is to form function arguments based on the specification provide in 'FUNCTIONS'.  Users will ask you a question about the weather, and you'll attempt to form the function call that we can pass along to a tool to determine the response. Do not try to answer questions about the weather directly, use the provided tools.  The tools are specified using a json schema format, which defines how the arguments to the function are to be formed.

FUNCTIONS = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "format": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"],
                    "description": "The temperature unit to use. Infer this from the users location.",
                },
            },
            "required": ["location", "format"],
        },
    }
]

Examples
###
Human: 
What's the weather like in Glasgow Scotland?
Assistant:
{
'function_call': {'name': 'get_current_weather',
    'arguments': {
        "location": "Glasgow, Scotland",
        "format": "celsius"
    }
}

Human: 
What's the weather like in Raleigh, NC?
Assistant:
{
'function_call': {'name': 'get_current_weather',
    'arguments': {
        "location": "Raleigh, North Carolina",
        "format": "fahrenheit"
    }
}
###

Human:
What's the weather like in Washington, DC?

Assistant:
"""

In [9]:
body = json.dumps({"prompt": prompt_data, "max_tokens_to_sample": 500})
modelId = "anthropic.claude-v2" 
accept = "application/json"
contentType = "application/json"

In [10]:
response = bedrock_runtime.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())

print(response_body.get("completion"))

 Here is the function call to get the current weather in Washington, DC:

{
'function_call': {
    'name': 'get_current_weather', 
    'arguments': {
        "location": "Washington, DC",
        "format": "fahrenheit"
    }
}


# And Now for the Main Event.

Here's a class that prompts a language model for function arguments for a linear regression, based on data that we load into the context from and external source.


In [11]:
from prompt_template import PromptTemplate, load_text_file
from function_parser import FunctionParser
from linear_trend_model import run_linear_trend_model, FUNCTIONS

In [12]:
#: Here's our prompt that we'll use to drive the language model
linear_trend_prompt = load_text_file('linear_trend_request.prompt')

#: Here's a mock article excerpt that I generated to showcase this technology.
city_dogs =  load_text_file('city_dogs.txt')


In [13]:
#: Establish the Prompt template from the text file.
linear_trend_prompt_template = PromptTemplate(linear_trend_prompt)

#: Fill the prompt using our 
linear_trend_filled_prompt = linear_trend_prompt_template.fill(
    FUNCTIONS=str(FUNCTIONS), 
    user_data=city_dogs
)
print(f"linear_trend_filled_prompt: {linear_trend_filled_prompt}")

linear_trend_filled_prompt: Human:

You will play the role of an expert Data Scientist assistant.  You love linear models, and when you get linear data you make a function call to fit a model to the data. We'll use triple hash ### to open and close subsections of prompt examples.

We're going to define your character and behaviour in terms of the red, yellow, blue, and green color categories that relate to personality types.

The color system I'm referring to - it's a way of categorizing personality traits into four colors:

Red personalities are usually seen as bold, expressive, and competitive. They are motivated by power, stimulation, and winning.
Yellow personalities are sociable, outgoing, and enthusiastic. They tend to be persuasive, verbal, and eager to collaborate.
Green personalities are calm, accommodating, and caring. They value stability, compassion, and building community.
Blue personalities are logical, detail-oriented, and focused. They are quantitative, deliberate, and 

In [14]:
body = json.dumps({"prompt": linear_trend_filled_prompt, "max_tokens_to_sample": 500})
modelId = "anthropic.claude-v2" 
accept = "application/json"
contentType = "application/json"

In [15]:
response = bedrock_runtime.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())

completion = response_body.get("completion")

print(f"completion:\n {completion}")

completion:
  Here is the function call with the extracted data:

{
  "function_call": {
    "name": "run_linear_trend_model",
    "parameters": {
      "x": [0.5, 1.5, 3, 4.2, 6], 
      "y": [15000, 12500, 10000, 7500, 6000]
    }
  }
}


In [16]:
type(completion)

str

In [17]:
parser = FunctionParser({'run_linear_trend_model': run_linear_trend_model})


In [18]:
completion

' Here is the function call with the extracted data:\n\n{\n  "function_call": {\n    "name": "run_linear_trend_model",\n    "parameters": {\n      "x": [0.5, 1.5, 3, 4.2, 6], \n      "y": [15000, 12500, 10000, 7500, 6000]\n    }\n  }\n}'

## Parsing an LLM's function call request.

The LLM is asking you to do something and return the value to it.  Here's where we parse the function parameters and submit the function calls.  The LLM can recieve these as inputs to more downstream processing, or return these results directly.

In our case, we've got some ordered data that consists of locations and revenue figures.

It's contained within a mock article about a popular NYC hot dog stand.  This example was contrived to illustrate the point of function calling.  The data was generated by a language model.

In [20]:
parser.parse_and_execute(completion)

x :[0.5, 1.5, 3, 4.2, 6]
y :[15000, 12500, 10000, 7500, 6000]
fit linear model in 0.004600048065185547 seconds.


"{'slope': -1652.7572364251002, 'intercept': 15224.381998732304, 'r_value': -0.9850169656708015, 'p_value': 0.0021966205939785346, 'std_err': 167.06548200469157}"